In [2]:
import numpy as np
%load_ext cython

## 扩展类型

### 扩展类型的基本结构

In [15]:
%%cython

cdef class Point2D:
    cdef public double x, y

In [9]:
%%language cpp
struct __pyx_obj_Point2D {
  PyObject_HEAD
  double x;
  double y;
};

In [13]:
print((type(Point2D.x)))
print((Point2D.x.__get__))
print((Point2D.x.__set__))

<type 'getset_descriptor'>
<method-wrapper '__get__' of getset_descriptor object at 0x097A3260>
<method-wrapper '__set__' of getset_descriptor object at 0x097A3260>


In [3]:
%%cython -a

cdef class Point2D:
    cdef public double x, y

cdef class Point3D(Point2D):
    cdef public double z
    
cdef Point3D p = Point3D()
p.x = 1.0
p.y = 2.0
p.z = 3.0

In [ ]:
%%language cpp
struct __pyx_obj_Point3D {
  struct __pyx_obj_Point2D __pyx_base;
  double z;
};

### 一维浮点数向量类型

In [1]:
%%include cython cython/vector.pyx 1
cdef class Vector:
    cdef int count
    cdef double * data

In [2]:
%%include cython cython/vector.pyx 2
    def __cinit__(self, data):
        cdef int i
        if isinstance(data, int):
            self.count = data
        else:
            self.count = len(data)
        self.data = <double *>mem.PyMem_Malloc(sizeof(double)*self.count)
        if self.data is NULL:
            raise MemoryError
        
        if not isinstance(data, int):
            for i in range(self.count):
                self.data[i] = data[i]

In [3]:
%%include cython cython/vector.pyx 3
    def __dealloc__(self):
        if self.data is not NULL:
            mem.PyMem_Free(self.data)

In [4]:
%%include cython cython/vector.pyx 4
    def __len__(self):
        return self.count
    
    cdef _check_index(self, int *index):
        if index[0] < 0:
            index[0] = self.count + index[0]
        if index[0] < 0  or index[0] > self.count - 1:
            raise IndexError("Vector index out of range")
    
    def __getitem__(self, int index):
        self._check_index(&index)
        return self.data[index]
    
    def __setitem__(self, int index, double value):
        self._check_index(&index)
        self.data[index] = value

In [5]:
%%include cython cython/vector.pyx 5
    def __add__(self, other):
        cdef Vector new, _self, _other

        if not isinstance(self, Vector): #❶
            self, other = other, self
        _self = <Vector>self #❸

        if isinstance(other, Vector): #❷       
            _other = <Vector>other
            if _self.count != _other.count:
                raise ValueError("Vector size not equal")
            new = Vector(_self.count) #❹
            add_array(_self.data, _other.data, new.data, _self.count)
            return new 
        new = Vector(_self.count)
        add_number(_self.data, <double>other, new.data, _self.count)
        return new

In [6]:
%%include cython cython/vector.pyx 6
    def __iadd__(self, other):
        cdef Vector _other
        if isinstance(other, Vector):
            _other = <Vector>other
            if self.count != _other.count:
                raise ValueError("Vector size not equal")
            add_array(self.data, _other.data, self.data, self.count)
        else:
            add_number(self.data, <double>other, self.data, self.count)
        return self

In [7]:
%%include cython cython/vector.pyx 7
    def __str__(self):
        values = ", ".join(str(self.data[i]) for i in range(self.count))
        norm = self.norm()
        return "Vector[{}]({})".format(norm, values)
    
    cpdef norm(self):
        cdef double *p
        cdef double s
        cdef int i
        s = 0
        p = self.data
        for i in range(self.count):
            s += p[i] * p[i]
        return s**0.5

In [8]:
%%include cython cython/vector.pyx 8
cdef add_array(double *op1, double *op2, double *res, int count):
    cdef int i
    for i in range(count):
        res[i] = op1[i] + op2[i]

cdef add_number(double *op1, double op2, double *res, int count):
    cdef int i
    for i in range(count):
        res[i] = op1[i] + op2

In [7]:
from scpy2.cython.vector import Vector
v1 = Vector(list(range(5)))
v2 = Vector(list(range(100, 105)))
print((len(v1)))
print((v1 + v2))
print((v1 + 2))
print((20 + v2))
print((v1.norm(), v2.norm()))
print([x**2 for x in v1])

5
Vector[232.637056378](100.0, 102.0, 104.0, 106.0, 108.0)
Vector[9.48683298051](2.0, 3.0, 4.0, 5.0, 6.0)
Vector[272.818621065](120.0, 121.0, 122.0, 123.0, 124.0)
5.47722557505 228.100854887
[0.0, 1.0, 4.0, 9.0, 16.0]


In [10]:
v1 = Vector(range(10000))
v2 = Vector(range(10000))
%timeit v1 + v2

a1 = np.arange(10000, dtype=float)
a2 = np.arange(10000, dtype=float)
%timeit a1 + a2

100000 loops, best of 3: 8.04 µs per loop
100000 loops, best of 3: 9.68 µs per loop


In [11]:
%timeit v1[100]
%timeit v1[100] = 2.0
%timeit a1[100]
%timeit a1[100] = 2.0

10000000 loops, best of 3: 62.2 ns per loop
10000000 loops, best of 3: 62.1 ns per loop
10000000 loops, best of 3: 122 ns per loop
10000000 loops, best of 3: 108 ns per loop


### 包装ahocorasick库

> **SOURCE**

> `scpy2.cython.multisearch`模块对C语言函数库`ahocorasick`进行包装。使用该模块可以快速在大量文本中同时搜索多个关键字。

In [12]:
from scpy2.cython import MultiSearch

ms = MultiSearch(["abc", "xyz"])
print((ms.isin("123abcdef")))
print((ms.isin("123uvwxyz")))
print((ms.isin("123456789")))

True
True
False


In [14]:
def process(pos, pattern):
    print(("found {0} at {1}".format(pattern, pos)))
    return 0

ms.search("123abc456xyz789abc", process)

found abc at 3
found xyz at 9
found abc at 15


In [15]:
for pos, pattern in ms.iter_search("123abc456xyz789abc"):
    print(("found {0} at {1}".format(pattern, pos)))

found abc at 3
found xyz at 9
found abc at 15


In [1]:
%%include c cython/ahocorasick/main.c 0
#include <stdio.h>
#include "ahocorasick.h"

/* 搜索关键字列表 */
AC_ALPHABET_t * allstr[] = {
    "recent", "from", "college"
};

#define PATTERN_NUMBER (sizeof(allstr)/sizeof(AC_ALPHABET_t *))

/* 搜索文本 */
AC_ALPHABET_t * input_text = {"She recently graduated from college"};

//*** 匹配时的回调函数
int match_handler(AC_MATCH_t * m, void * param)
{
    unsigned int j;

    printf ("@ %ld : %s\n", m->position, m->patterns->astring);
    /* 返回0继续搜索，返回1停止搜索 */
    return 0;
}

int main (int argc, char ** argv)
{
    unsigned int i;

    AC_AUTOMATA_t * acap;
    AC_PATTERN_t tmp_patt;
    AC_TEXT_t tmp_text;

    //*** 创建AC_AUTOMATA_t结构体，并传递回调函数
    acap = ac_automata_init();

    //*** 添加关键字
    for (i=0; i<PATTERN_NUMBER; i++)
    {
        tmp_patt.astring = allstr[i];
        tmp_patt.rep.number = i+1; // optional
        tmp_patt.length = strlen(tmp_patt.astring);
        ac_automata_add (acap, &tmp_patt);
    }

    //*** 结束添加关键字
    ac_automata_finalize (acap);

    //*** 设置待搜索字符串
    tmp_text.astring = input_text;
    tmp_text.length = strlen(tmp_text.astring);

    //*** 搜索
    ac_automata_search (acap, &tmp_text, 0, match_handler, NULL);

    //*** 释放内存
    ac_automata_release (acap);
    return 0;
}

In [2]:
%%include cython cython/multisearch.pyx 1
cdef extern from "ahocorasick.h": #❶
    ctypedef int (*AC_MATCH_CALBACK_f)(AC_MATCH_t *, void *) #❷
    ctypedef enum AC_STATUS_t: #❸
        ACERR_SUCCESS = 0
        ACERR_DUPLICATE_PATTERN
        ACERR_LONG_PATTERN
        ACERR_ZERO_PATTERN
        ACERR_AUTOMATA_CLOSED

    ctypedef struct AC_MATCH_t: #❹
        AC_PATTERN_t * patterns
        long position
        unsigned int match_num

    ctypedef struct AC_AUTOMATA_t:
        AC_MATCH_t match

    ctypedef struct AC_PATTERN_t:
        char * astring
        unsigned int length

    ctypedef struct AC_TEXT_t:
        char * astring
        unsigned int length

    #❺
    AC_AUTOMATA_t * ac_automata_init() 
    AC_STATUS_t ac_automata_add(AC_AUTOMATA_t * thiz, AC_PATTERN_t * pattern)
    void ac_automata_finalize(AC_AUTOMATA_t * thiz)
    int ac_automata_search(AC_AUTOMATA_t * thiz, AC_TEXT_t * text, int keep, 
        AC_MATCH_CALBACK_f callback, void * param)
    void ac_automata_settext (AC_AUTOMATA_t * thiz, AC_TEXT_t * text, int keep)
    AC_MATCH_t * ac_automata_findnext (AC_AUTOMATA_t * thiz)        
    void ac_automata_release(AC_AUTOMATA_t * thiz)

In [3]:
%%include cython cython/multisearch.pyx 2
cdef class MultiSearch:
    
    cdef AC_AUTOMATA_t * _auto #❶
    cdef bint found
    cdef object callback
    cdef object exc_info

    def __cinit__(self, keywords):
        self._auto = ac_automata_init()
        if self._auto is NULL:
            raise MemoryError
        self.add(keywords) #❷

    def __dealloc__(self):
        if self._auto is not NULL:
            ac_automata_release(self._auto)
            
    cdef add(self, keywords):
        cdef AC_PATTERN_t pattern
        cdef bytes keyword
        cdef AC_STATUS_t err
        
        for keyword in keywords: #❸
            pattern.astring = <char *>keyword
            pattern.length = len(keyword)
            err = ac_automata_add(self._auto, &pattern)
            if err != ACERR_SUCCESS:
                raise ValueError("Error Code:%d" % err)

        ac_automata_finalize(self._auto)

In [9]:
%%include cython cython/multisearch.pyx 3
    def isin(self, bytes text, bint keep=False):
        cdef AC_TEXT_t temp_text   #❶
        temp_text.astring = <char *>text
        temp_text.length = len(text)
        self.found = False         #❷
        ac_automata_search(self._auto, &temp_text, keep, isin_callback, <void *>self) #❸
        return self.found

In [10]:
%%include cython cython/multisearch.pyx 4
cdef int isin_callback(AC_MATCH_t * match, void * param):
    cdef MultiSearch ms = <MultiSearch> param #❶
    ms.found = True  #❷
    return 1  #❸

> **TIP**

> 为了介绍C语言回调函数和Python回调函数的用法，这里使用`ac_automata_search()`。实际上使用后面介绍的`ac_automata_findnext()`可以更方便地编写`isin()`和`search()`函数。

In [11]:
%%include cython cython/multisearch.pyx 5
    def search(self, bytes text, callback, bint keep=False):
        cdef AC_TEXT_t temp_text
        temp_text.astring = <char *>text
        temp_text.length = len(text)
        self.found = False
        self.callback = callback  #❶
        self.exc_info = None
        ac_automata_search(self._auto, &temp_text, keep, search_callback, <void *>self) #❷
        if self.exc_info is not None:
            raise self.exc_info[1], None, self.exc_info[2]  #❸

In [12]:
%%include cython cython/multisearch.pyx 6
cdef int search_callback(AC_MATCH_t * match, void * param):
    cdef MultiSearch ms = <MultiSearch> param
    cdef bytes pattern = match.patterns.astring
    cdef int res = 1
    try:
        res = ms.callback(match.position - len(pattern), pattern)  #❶
    except Exception as ex:
        import sys
        ms.exc_info = sys.exc_info()  #❷
    return res

In [13]:
%%include cython cython/multisearch.pyx 7
    def iter_search(self, bytes text, bint keep=False):
        cdef AC_TEXT_t temp_text
        cdef AC_MATCH_t * match
        cdef bytes matched_pattern
        temp_text.astring = <char *>text
        temp_text.length = len(text)
        ac_automata_settext(self._auto, &temp_text, keep)
        while True:
            match = ac_automata_findnext(self._auto)
            if match == NULL:
                break
            matched_pattern = <bytes>match.patterns.astring
            yield match.position - len(matched_pattern), matched_pattern